In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=b1c94684ccd56df7311dccdf6e750b793a3bd3572261f40cdd80a0f1d46be864
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession, SQLContext

In [ ]:
spark = SparkSession.builder.appName('Movie Recommendation').getOrCreate()

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

lines = spark.read.text("/content/drive/MyDrive/Colab Notebooks/Bigdata/Week 15/ratings.dat").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=int(p[2]), timestamp=int(p[3])))

In [ ]:
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
predictions.show()

import math
result = predictions.rdd.map(lambda row: row['prediction'] - row['rating']).map(lambda x: x*x).filter(lambda x: not math.isnan(x))
mse = result.reduce(lambda x,y: x+y)


+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|    260|     4|978300760| 4.3861814|
|     1|    661|     3|978302109| 2.8327825|
|     1|    919|     4|978301368| 4.7771387|
|     1|   1029|     5|978302205| 4.4698286|
|     1|   1035|     5|978301753|  4.160609|
|     1|   1545|     4|978824139| 3.1587648|
|     1|   1721|     4|978300055| 5.1249423|
|     1|   2687|     3|978824268| 3.8394065|
|     2|    349|     4|978299839| 3.7561812|
|     2|    442|     3|978300025| 2.6571643|
|     2|    589|     4|978299773| 3.5257268|
|     2|    590|     5|978299083| 4.4513745|
|     2|    780|     3|978299966| 3.9276054|
|     2|    902|     2|978298905| 4.3690367|
|     2|    982|     4|978299269|  3.883231|
|     2|   1084|     3|978298813| 4.0071673|
|     2|   1096|     4|978299386| 3.5872035|
|     2|   1193|     5|978298413| 4.3087916|
|     2|   1198|     4|978298124|  4.358396|
|     2|  